# PARTE 1 DE CREANDO EL RAG
**(PUSE TODO COMO EN PARTES, PORQUE ME QUEDA MAS FACIL MENTALMENTE ORGANIZARME ASI)**

In [1]:
# ============================================
# 2. CARGAR CLAVES
# ============================================

import os
import getpass

# Activar trazado de LangSmith
os.environ["LANGSMITH_TRACING"] = "true"

# Configurar LangSmith API Key
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Ingresa tu API key de LangSmith: ")

print("LangSmith configurado correctamente.")

# Configurar Pinecone API Key
if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Ingresa tu Pinecone API Key: ")

print("Pinecone configurado correctamente.")


LangSmith configurado correctamente.
Pinecone configurado correctamente.


In [3]:
!{sys.executable} -m pip install pinecone-client==2.2.4


In [5]:
# -----------------------------
# 3. Conectar al índice existente (Pinecone serverless + HuggingFace gratis)
# -----------------------------
import sys
!{sys.executable} -m pip uninstall -y pinecone pinecone-client langchain-pinecone
!{sys.executable} -m pip install pinecone-client==2.2.4
!{sys.executable} -m pip install -U langchain-community sentence-transformers

import pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

# Inicializar cliente Pinecone
pinecone.init(
    api_key=os.environ["PINECONE_API_KEY"],
    host="https://rags-pkz6jud.svc.aped-4627-b74a.pinecone.io"
)

# Conectar al índice existente
index = pinecone.Index("rags")

# Embeddings gratuitos de HuggingFace (384 dimensiones)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Vector store usando el índice existente
vector_store = PineconeVectorStore(
    embedding=embeddings,
    index=index,
    text_key="text"
)

print("Conectado correctamente al índice 'rags'.")


Found existing installation: pinecone-client 2.2.4
Uninstalling pinecone-client-2.2.4:
  Successfully uninstalled pinecone-client-2.2.4


  Using cached pinecone_client-2.2.4-py3-none-any.whl.metadata (7.8 kB)
Using cached pinecone_client-2.2.4-py3-none-any.whl (179 kB)
Conectado correctamente al índice 'rags'.


In [6]:
# ============================================
# 4. Cargar docs de la web
# ============================================
import sys
!{sys.executable} -m pip install -U bs4 langchain-community

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Definir el filtro de BeautifulSoup para extraer solo ciertas clases
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

# Cargar documento desde la web
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

# Validar que se cargó un documento
assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

# Mostrar una vista previa del contenido
print("\n--- Preview del contenido ---\n")
print(docs[0].page_content[:500])


USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047

--- Preview del contenido ---



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [7]:
# ============================================
# 5. Hacer SPLIT
# ============================================
import sys
!{sys.executable} -m pip install -U langchain-text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Crear el splitter con parámetros de chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)

# Dividir los documentos cargados en sub-documentos
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")


Split blog post into 63 sub-documents.


In [10]:
# ============================================
# 6. Indexar en Pinecone
# ============================================

document_ids = vector_store.add_documents(documents=all_splits)
print(document_ids[:3])



ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

# PARTE 2: Retrieval and Generation

In [13]:
# ============================================
# 7. RAG AGENT (versión con herramientas)
# ============================================
import sys
!{sys.executable} -m pip install -U langchain langchain-community langchain-openai

from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, tool

# Modelo base (puedes ajustar a gpt-4o-mini o el que prefieras)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Definición de la herramienta de recuperación de contexto
@tool
def retrieve_context(query: str) -> str:
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized

tools = [retrieve_context]

# Crear el agente RAG con herramientas
agent = initialize_agent(
    tools,
    model,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

print("Agente RAG creado correctamente.")

# Consulta de ejemplo
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

response = agent.run(query)
print(response)


ImportError: cannot import name 'initialize_agent' from 'langchain.agents' (d:\Users\solos\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain\agents\__init__.py)

In [14]:
# ============================================
# 8. RAG CHAIN DINÁMICO (middleware)
# ============================================
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Prompt dinámico que inyecta contexto desde el vector_store
def build_prompt_with_context(query: str) -> str:
    retrieved_docs = vector_store.similarity_search(query, k=3)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )
    return system_message

# Crear prompt template dinámico
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "{context}"),
    ("user", "{question}")
])

# Crear chain con el modelo
rag_chain = LLMChain(llm=model, prompt=prompt_template)

# Consulta de ejemplo
query = "What is task decomposition?"
context = build_prompt_with_context(query)

response = rag_chain.run({"context": context, "question": query})
print(response)


ModuleNotFoundError: No module named 'langchain.chains'